<a href="https://colab.research.google.com/github/Jeromeschmidt/DS-2.2-Deep-Learning/blob/master/labs/Keras_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [10]:
from keras.utils import to_categorical

X_train = X_train.reshape(60000,28,28,1)

X_test = X_test.reshape(10000,28,28,1)

y_train = to_categorical(y_train)

y_test = to_categorical(y_test)

y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [14]:
from keras.models import Sequential

from keras.layers import Dense, Conv2D, Flatten

model = Sequential()

model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))

model.add(Conv2D(32, kernel_size=3, activation='relu'))

model.add(Flatten())

model.add(Dense(10, activation='softmax'))

In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

Epoch 1/3
1875/1875 [==============================] - 169s 90ms/step - loss: 0.2337 - accuracy: 0.9523 - val_loss: 0.0936 - val_accuracy: 0.9704
Epoch 2/3
1875/1875 [==============================] - 174s 93ms/step - loss: 0.0702 - accuracy: 0.9789 - val_loss: 0.0759 - val_accuracy: 0.9756
Epoch 3/3
1875/1875 [==============================] - 169s 90ms/step - loss: 0.0464 - accuracy: 0.9857 - val_loss: 0.1020 - val_accuracy: 0.9734


In [17]:
model.predict(X_test[:4])

y_test[:4]

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [18]:
import skimage.data

img = skimage.data.chelsea()

img = skimage.color.rgb2gray(img)

In [19]:
l1_filter = numpy.zeros((2,3,3))
l1_filter[0, :, :] = numpy.array([[[-1, 0, 1],   
                                   [-1, 0, 1],   
                                   [-1, 0, 1]]])  
l1_filter[1, :, :] = numpy.array([[[1,   1,  1],   
                                   [0,   0,  0],   
                                   [-1, -1, -1]]])

NameError: ignored

In [ ]:
l1_feature_map = conv(img, l1_filter)

In [ ]:
def conv(img, conv_filter):  
    if len(img.shape) > 2 or len(conv_filter.shape) > 3: 
    if img.shape[-1] != conv_filter.shape[-1]:  
            print("Error: Number of channels in both image and filter must match.")  
            sys.exit()  
    if conv_filter.shape[1] != conv_filter.shape[2]: 
        print('Error: Filter must be a square matrix. I.e. number of rows and columns must match.')  
        sys.exit()  
    if conv_filter.shape[1]%2==0: 
        print('Error: Filter must have an odd size. I.e. number of rows and columns must be odd.')  
        sys.exit()

In [ ]:
   feature_maps = numpy.zeros((img.shape[0]-conv_filter.shape[1]+1,   
                               img.shape[1]-conv_filter.shape[1]+1,   
                               conv_filter.shape[0]))  

  for filter_num in range(conv_filter.shape[0]):  
       print("Filter ", filter_num + 1)  
       curr_filter = conv_filter[filter_num, :]
       if len(curr_filter.shape) > 2:  
           conv_map = conv_(img[:, :, 0], curr_filter[:, :, 0]) 
           for ch_num in range(1, curr_filter.shape[-1]):
               conv_map = conv_map + conv_(img[:, :, ch_num],   
                                 curr_filter[:, :, ch_num])  
       else: 
            conv_map = conv_(img, curr_filter)  
       feature_maps[:, :, filter_num] = conv_map
    return feature_maps

In [ ]:
l1_feature_map_relu = relu(l1_feature_map)

In [ ]:
def relu(feature_map):  
   #Preparing the output of the ReLU activation function.  
   relu_out = numpy.zeros(feature_map.shape)  
   for map_num in range(feature_map.shape[-1]):  
       for r in numpy.arange(0,feature_map.shape[0]):  
           for c in numpy.arange(0, feature_map.shape[1]):  
               relu_out[r, c, map_num] = numpy.max(feature_map[r, c, map_num], 0)

In [ ]:
l1_feature_map_relu_pool = pooling(l1_feature_map_relu, 2, 2)

In [ ]:
def pooling(feature_map, size=2, stride=2):  
    pool_out = numpy.zeros((numpy.uint16((feature_map.shape[0]-size+1)/stride),  
                            numpy.uint16((feature_map.shape[1]-size+1)/stride),  
                            feature_map.shape[-1]))  
    for map_num in range(feature_map.shape[-1]):  
        r2 = 0  
        for r in numpy.arange(0,feature_map.shape[0]-size-1, stride):  
            c2 = 0  
            for c in numpy.arange(0, feature_map.shape[1]-size-1, stride):  
                pool_out[r2, c2, map_num] = numpy.max(feature_map[r:r+size,  c:c+size])  
                c2 = c2 + 1  
                r2 = r2 +1 

In [ ]:
l2_filter = numpy.random.rand(3, 5, 5, l1_feature_map_relu_pool.shape[-1])  
l2_feature_map = conv(l1_feature_map_relu_pool, l2_filter)  
l2_feature_map_relu = relu(l2_feature_map)  
l2_feature_map_relu_pool = pooling(l2_feature_map_relu, 2, 2)  

l3_filter = numpy.random.rand(1, 7, 7, l2_feature_map_relu_pool.shape[-1])  
l3_feature_map = conv(l2_feature_map_relu_pool, l3_filter)  
l3_feature_map_relu = relu(l3_feature_map)  
l3_feature_map_relu_pool = pooling(l3_feature_map_relu, 2, 2)